<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Exercise 6.2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load Excel file
fileName = 'PV Modeling Book Data Source.xls'
sheetName = 'Source 7'
llp = 0.01
LOAD = 1

# (1) Assumptions
PV_eff = 0.16
Inv_eff = 0.90
V_B = 12
DOD = 0.8
Charge_eff = 0.8
Wp_Cost = 3.7
Ah_Cost = 2.5
kWp_Cost = 1200

# (3) Energy sources size
SE = pd.read_excel(fileName, sheet_name=sheetName, usecols='K', skiprows=1, nrows=365).values.flatten()
WS = pd.read_excel(fileName, sheet_name=sheetName, usecols='J', skiprows=1, nrows=365).values.flatten()

C_A = []
C_W = []
C_D = []
LLP = []
C_B = []

for PV_A in np.arange(0, 1.05, 0.05):
    PV_E = 0.17 * SE * PV_A * 0.9
    C_Ai = (PV_A * 0.17 * 4.9 * 0.9) / LOAD

    for R in np.arange(0, 1.05, 0.05):
        Air_Density = 1.22521
        W_Ei = (Air_Density * 3.14 * (R ** 2) * 0.5 * 24 * (WS ** 3)) / 1000
        W_E = W_Ei * 0.2
        C_Wi = np.sum(W_E) / (LOAD * 365)

        for E_diesel in np.arange(0, 1.05, 0.05):
            C_Di = E_diesel / LOAD
            C_D.append(C_Di)
            C_A.append(C_Ai)
            C_W.append(C_Wi)

            E_T = PV_E + W_E + E_diesel
            E_NET = E_T - LOAD

            EN = E_NET[E_NET < 0]
            EP = E_NET[E_NET >= 0]

            C_Bi = (np.sum(EP) * 0.8 / 365) / LOAD
            C_B.append(C_Bi)

            LLPi = (-1 * np.sum(EN)) / (LOAD * 365)
            LLP.append(LLPi)

C_A = np.array(C_A)
C_W = np.array(C_W)
C_D = np.array(C_D)
C_B = np.array(C_B)
LLP = np.array(LLP)

Array = np.column_stack((C_A, C_W, C_D, C_B, LLP))

llp_modified1 = llp + 0.0003
llp_modified2 = llp - 0.0003

Array_New = []

for j in range(len(Array)):
    if llp_modified2 <= Array[j, 4] <= llp_modified1:
        Array_New.append(Array[j])

Array_New = np.array(Array_New)

if Array_New.shape[0] > 0:
    T = np.arange(0, len(Array_New))
    x_1 = Array_New[T, 0]
    x_2 = Array_New[T, 1]
    x_3 = Array_New[T, 2]
    x_4 = Array_New[T, 3]
    x_5 = x_1 + x_2 + x_3

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot3D(x_1, x_2, x_3, 'b.')
    ax.set_xlabel('PV')
    ax.set_ylabel('Wind')
    ax.set_zlabel('Diesel')
    plt.title('PV-Wind-Diesel Combination')
    plt.show()
else:
    print("No configurations found within LLP range.")


ParserError: Defining usecols with out-of-bounds indices is not allowed. [10] are out-of-bounds. (sheet: Source 7)